<a href="https://colab.research.google.com/github/Liguilin-98/Git_Projects/blob/master/3AttentionLstmDemo3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 所有导入的包

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.serialization import load

from torch.utils.data import DataLoader

import os
from glob import glob
import random
import time
import datetime

from tqdm import tqdm

import numpy as np
import pandas as pd
from IPython.display import FileLink
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns 
%matplotlib inline
from IPython.display import display, Image
import matplotlib.image as mpimg
import cv2

import itertools 

import torch.optim as optim
from torchvision.datasets import ImageFolder

from torch.utils.data import random_split
from torch.utils.data import Dataset,DataLoader
from torchvision.utils import make_grid
import torch.nn.functional as F
from torchvision import models
from tqdm.notebook import tqdm

In [ ]:
print(torch.__version__)
print(cv2.__version__)

1.11.0+cu113
4.1.2


# 从kaggle导入数据集

In [ ]:
!pip install kaggle

import json
token = {"username":"guilinlie","key":"af8933a0ec6dc42a577c0143979119a3"}
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)

!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content

!kaggle competitions download -c state-farm-distracted-driver-detection
#上面这些步骤无论在kaggle上下载什么数据集都需要重复的
#https://blog.csdn.net/qq_20880939/article/details/105613800
#

!unzip /content/competitions/state-farm-distracted-driver-detection/state-farm-distracted-driver-detection.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
- path is now set to: /content
state-farm-distracted-driver-detection.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/competitions/state-farm-distracted-driver-detection/state-farm-distracted-driver-detection.zip
replace driver_imgs_list.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# 数据预处理


*   读取数据
*   裁剪数据



In [ ]:
#Classes
NUMBER_CLASSES = 10


activity_map = {'c0': 'Safe driving', 
                'c1': 'Texting - right', 
                'c2': 'Talking on the phone - right', 
                'c3': 'Texting - left', 
                'c4': 'Talking on the phone - left', 
                'c5': 'Operating the radio', 
                'c6': 'Drinking', 
                'c7': 'Reaching behind', 
                'c8': 'Hair and makeup', 
                'c9': 'Talking to passenger'}
# Train path
train_path = "/content/imgs/train"
# Test path
test_path = "/content/imgs/test"

train_length = 0
for clss in os.listdir(train_path):
    print("%s size: %d" % (clss, len(os.listdir(os.path.join(train_path, clss)))))
    train_length += len(os.listdir(os.path.join(train_path, clss)))
print("Train size: %d" % train_length)
print("Test Size: %d" % len(os.listdir(test_path)))

c0 size: 2489
c6 size: 2325
c2 size: 2317
c9 size: 2129
c1 size: 2267
c7 size: 2002
c3 size: 2346
c4 size: 2326
c8 size: 1911
c5 size: 2312
Train size: 22424
Test Size: 79726


In [ ]:
# Data Transforms and Augmentation
train_transforms = transforms.Compose([ transforms.Resize((64, 64)),
                               transforms.Grayscale(num_output_channels=1),
                               transforms.RandomAdjustSharpness(2),
                               transforms.RandomRotation((-15, 15)),
                               transforms.ColorJitter(brightness=.5, hue=.3),
                               transforms.ToTensor()
                             ])

# Loading Data using ImageFolder
train_ds = ImageFolder(train_path, train_transforms)
classes = train_ds.classes
print(classes)

# Splitting into train-val set
val_pct = .1
val_size = int(val_pct * len(train_ds))
train_ds ,valid_ds = random_split(train_ds, [len(train_ds)-val_size, val_size])

# Data Loader
batch_size = 64
train_loader = DataLoader(train_ds, batch_size, shuffle = True, num_workers=2, pin_memory=True)
test_loader = DataLoader(valid_ds, batch_size, num_workers=2, pin_memory=True)


['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']


In [ ]:
print('Number of train samples: ', len(train_ds))
img, target = train_ds[3] # load 4th sample

print("Image Size: ", img.size())
print(target)

print('Number of test samples: ', len(valid_ds))

n_iters = 30000
num_epochs = n_iters / (len(train_ds) / batch_size)
num_epochs = int(num_epochs)
print("num_epoch",num_epochs)

Number of train samples:  20182
Image Size:  torch.Size([1, 64, 64])
4
Number of test samples:  2242
num_epoch 95


输入输出的各个维度含义为 (seq_length,batch,feature)
序列长度，batch，特征
长，batch，宽
batchfirst=true表示把batch放在第一位，batch，长，宽

batch_first=False的情况下
*   inputlayer_size(seq_len, batch, input_size)
*   h0_size(num_layers * num_directions, batch, hidden_size)
*   c0_size(num_layers * num_directions, batch, hidden_size)
*   outlayer_size(seq_len, batch, num_directions * hidden_size)
*   



*   batch_first=True的情况输出
*   torch.Size([100, 3, 32, 32])//print(image.shape)
*   torch.Size([300, 32, 32])//print(x.shape)
*   torch.Size([1, 300, 100])//print(h0.shape)
*   torch.Size([1, 300, 100])//print(c0.shape)
*   torch.Size([300, 32, 100])//print(out.shape)
*   torch.Size([1, 300, 100])//print(hn.shape)
*   torch.Size([1, 300, 100])//print(cn.shape)
*   torch.Size([300, 10])//print(out.shape)


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        # Hidden dimensions
        
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim
        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def attention(self, lstm_output, final_state):
        #print(lstm_output.size()) = (batch_size, squence_length, hidden_size*layer_size)
        #torch.Size([100, 32, 100])//print(out.shape)
        #hn_size(num_layers * num_directions, batch, hidden_size)
        #torch.Size([1, 100, 100])//print(hn.shape)        
        hidden = final_state.permute(1, 2, 0)#[batch_size, n_hidden * num_directions, n_layer)] [100,100,1]       
        attn_weights = torch.bmm(lstm_output, hidden).squeeze(2) 
        #attn_weights : [batch_size, n_hidden * num_directions] [100,32]
        #假定两个张量的形状分别是(n,a,b)和(n,b,c)， 它们的批量矩阵乘法输出的形状为(n,a,c)。
        # .squeeze(2)表示若第三维度为1则去掉第三维度
        #attn_tanh = torch.tanh(attn_weights)
        #print(attn_tanh.size()) = (squence_length * batch_size, attention_size)
        soft_attn_weights = F.softmax(attn_weights, 1)#按列方向上进行softmax
        # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
        new_hidden_state = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
        return new_hidden_state


    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        #print(x.shape)
        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        #print(h0.shape)
        #print(c0.shape)
        # 28 time steps
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        attn_output = self.attention(out,hn)
        #print(out.shape) # torch.Size([300, 32, 100])        
        #print(hn.shape)
        #print(hn.shape)
        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(attn_output.squeeze(0))
        
        # out.size() --> 100, 10
        #print(out.shape)
        return out

In [ ]:
input_dim = 64
hidden_dim = 100
layer_dim = 1
output_dim = 10

In [ ]:
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

In [ ]:
criterion = nn.CrossEntropyLoss()
learning_rate = 1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 
len(list(model.parameters()))

6

In [ ]:
for i in range(len(list(model.parameters()))):
  print(list(model.parameters())[i].size())

torch.Size([400, 64])
torch.Size([400, 100])
torch.Size([400])
torch.Size([400])
torch.Size([10, 100])
torch.Size([10])


*   images.size()= torch.Size([100, 1, 32, 32]) print('images.size()=',images.size())
*   torch.Size([100, 32, 32])
*   torch.Size([1, 100, 100])
*   torch.Size([1, 100, 100])
*   torch.Size([100, 32, 100])
*   torch.Size([1, 100, 100])
*   torch.Size([1, 100, 100])
*   torch.Size([100, 10])

In [ ]:
# Number of steps to unroll
seq_dim = 64  
batch = 100
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as a torch tensor with gradient accumulation abilities
        #print('images.size()=',images.size())
        images = images.view(-1, seq_dim, input_dim).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Resize images
                images = images.view(-1, seq_dim, input_dim)
                #images = images.view(100, -1, -1)
                # Forward pass only to get logits/output
                outputs = model(images)


                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 2.085143566131592. Accuracy: 22.033897399902344
Iteration: 1000. Loss: 1.7504806518554688. Accuracy: 34.12131881713867
Iteration: 1500. Loss: 1.2793322801589966. Accuracy: 46.16413879394531
Iteration: 2000. Loss: 0.8955131769180298. Accuracy: 59.99108123779297
Iteration: 2500. Loss: 0.9913192987442017. Accuracy: 71.94469451904297
Iteration: 3000. Loss: 0.5992252230644226. Accuracy: 77.78768920898438
Iteration: 3500. Loss: 0.4769154191017151. Accuracy: 82.2033920288086
Iteration: 4000. Loss: 0.23884396255016327. Accuracy: 83.63068389892578
Iteration: 4500. Loss: 0.4033939242362976. Accuracy: 83.71989440917969
Iteration: 5000. Loss: 0.7220348715782166. Accuracy: 87.86797332763672
Iteration: 5500. Loss: 0.41306251287460327. Accuracy: 85.14718627929688
Iteration: 6000. Loss: 0.2748153507709503. Accuracy: 88.40321350097656
Iteration: 6500. Loss: 0.2348773181438446. Accuracy: 91.92684936523438
Iteration: 7000. Loss: 0.15133889019489288. Accuracy: 91.48081970214844
Itera

In [ ]:
print(outputs.size())
print(images.size())
print(labels.size())

torch.Size([22, 10])
torch.Size([22, 64, 64])
torch.Size([22])


In [ ]:
print(model)
torch.save(model.state_dict(), 'model_dict.pt')
torch.save(model,'model_all.pt')

In [ ]:
# 更新混淆矩阵
def confusion_matrix(preds, labels, conf_matrix):
    for p, t in zip(preds, labels):
        conf_matrix[p, t] += 1
    return conf_matrix

In [ ]:

'''
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Input
    - cm : 计算出的混淆矩阵的值
    - classes : 混淆矩阵中每一行每一列对应的列
    - normalize : True:显示百分比, False:显示个数
'''

# 绘制混淆矩阵
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):

  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Normalized confusion matrix")
    
  else:
    print('Confusion matrix, without normalization')

  print(cm)
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=90)
  plt.yticks(tick_marks, classes)
  
# 。。。。。。。。。。。。新增代码开始处。。。。。。。。。。。。。。。。
# x,y轴长度一致(问题1解决办法）
  plt.axis("equal")
  # x轴处理一下，如果x轴或者y轴两边有空白的话(问题2解决办法）
  ax = plt.gca()  # 获得当前axis
  left, right = plt.xlim()  # 获得x轴最大最小值
  ax.spines['left'].set_position(('data', left))
  ax.spines['right'].set_position(('data', right))
  for edge_i in ['top', 'bottom', 'right', 'left']:
      ax.spines[edge_i].set_edgecolor("white")
# 。。。。。。。。。。。。新增代码结束处。。。。。。。。。。。。。。。。

  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      num = '{:.2f}'.format(cm[i, j]) if normalize else int(cm[i, j])
      plt.text(i, j, num,
                verticalalignment='center',
                horizontalalignment="center",
                color="white" if num > thresh else "black")
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

In [ ]:
# 创建一个空矩阵存储混淆矩阵
conf_matrix = torch.zeros(10, 10)
for batch_images, batch_labels in test_loader:
   print(batch_labels)
   print(batch_images.size)
   batch_images = batch_images.view(-1, 64, 64).requires_grad_()


   out = model(batch_images)

   prediction = torch.max(out, 1)[1]
   conf_matrix = confusion_matrix(prediction, labels=batch_labels, conf_matrix=conf_matrix)

# conf_matrix需要是numpy格式
# attack_types是分类实验的类别，eg：attack_types = ['Normal', 'DoS', 'Probe', 'R2L', 'U2R']
attack_types = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']
plot_confusion_matrix(conf_matrix.numpy(), classes=attack_types, normalize=False,
                                 title='Normalized confusion matrix')
